**Задача поиск похожих по эмбеддингам**
Скачиваем датасет 

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

```
# пример работы с ним 
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)
```


что надо сделать 

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [17]:
!pip install natasha stop_words annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import string
import pickle
import annoy
import numpy as np
import pandas as pd

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

In [19]:
!tar -xzvf gazeta_jsonl.tar.gz

tar (child): gazeta_jsonl.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [20]:
# Создадим датафрейм на основе всех доступных данных.
df_train = pd.read_json(path_or_buf='/content/gazeta_train.jsonl', lines=True)
df_val = pd.read_json(path_or_buf='/content/gazeta_val.jsonl', lines=True)
df_test = pd.read_json(path_or_buf='/content/gazeta_test.jsonl', lines=True)

df = pd.concat([df_train, df_val, df_test], ignore_index=True)
df.head()

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01


In [21]:
# В качестве основы для обучения моделей будет использовать столбец с саммари статей, 
# так как он отражает общую суть материалов, но при этом значительно короче, а значит обучение будет

In [22]:
text_lengths = df['text'].apply(len)
summary_lengths = df['summary'].apply(len)

print(f'Средняя длина текста статьи - {np.average(text_lengths):.0f} слов.')
print(f'Средняя длина саммари - {np.average(summary_lengths):.0f} слов.')

Средняя длина текста статьи - 4519 слов.
Средняя длина саммари - 312 слов.


In [23]:
morpher = MorphAnalyzer()
stopwords = set(get_stop_words("ru"))
punkt = set(string.punctuation)

def preprocess_txt(line, morpher=morpher, sw=stopwords, punkt=punkt):
    spls = "".join(i for i in line.strip() if i not in punkt).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

df['preprocessed_summary'] = df['summary'].apply(preprocess_txt)
df.head()

,url,text,title,summary,date,preprocessed_summary
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39,"[2011, россия, уйти, 80, млрд, считать, минэко..."
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09,"[юлия, соловьёв, эксдиректор, холдинг, «профме..."
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14,"[суд, арестовать, месяц, четверо, эксчиновник,..."
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50,"[мнение, предприниматель, повод, вступление, в..."
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01,"[канал, завершиться, сезон, шоу, «голос», побе..."


In [24]:
# Создадим модели на основе массива предобработанных саммари статей

In [28]:
window_size = 5
vector_size = 300

modelW2V = Word2Vec(sentences=df['preprocessed_summary'], 
                    size=vector_size,
                    window=window_size, 
                    min_count=1
                    )
modelFT = FastText(sentences=df['preprocessed_summary'], 
                   size=vector_size, 
                   window=window_size, 
                   min_count=1
                   )

In [29]:
w2v_index = annoy.AnnoyIndex(vector_size, 'angular')
ft_index = annoy.AnnoyIndex(vector_size, 'angular')

index_map = {}
counter = 0


for i in range(len(df['preprocessed_summary'])):
    n_w2v = 0
    n_ft = 0
    index_map[i] = df['text'][i]
    
    vector_w2v = np.zeros(vector_size)
    vector_ft = np.zeros(vector_size)
    for word in df['preprocessed_summary'][i]:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(i, vector_w2v)
    ft_index.add_item(i, vector_ft)


w2v_index.build(10)
ft_index.build(10)

True

In [30]:
def get_response(question, index, model, index_map, nn=5):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, nn, )
    return [index_map[i] for i in answers]

In [31]:
TEXT = "Футбольные фанаты устроили беспорядки"

In [32]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Бывший капитан « Манчестер Юнайтед » Уэйн Руни неплохо проводит время в перерыве на матчи сборных. В конце августа 31-летний нападающий объявил о завершении карьеры в национальной команде Англии, так что матчи отборочного турнира к чемпионату мира – 2018 в России ему не грозят. Вместо этого игрок, вернувшийся в этом сезоне в родной «Эвертон», посещает вечеринки в графстве Чешир, где у него есть дом за £6 млн, и не ограничивает себя в приеме алкоголя. Так, вечером 31 августа Руни был задержан за рулем собственного автомобиля по подозрению в вождении в нетрезвом виде. Футболист подъезжал к своему коттеджу, когда полицейские заметили что-то странное в его поведении и попросили остановиться. После быстрого разговора стражи правопорядка настояли на том, чтобы Руни проследовал с ними в участок. После еще одного допроса лучший бомбардир в истории сборной Англии был оставлен на ночь под присмотром полицейских, и, по информации Daily Mail, еще не вернулся домой. Накануне форвард «Эвертона» по

In [33]:
get_response(TEXT, ft_index, modelFT, index_map)

['Юниорский турнир по вольной борьбе, посвященный памяти трагически погибшего призера Олимпийских игр Бесика Кудухова, был омрачен массовой дракой. Стычка началась между одним из борцов и тренером его соперника. Как ни странно, инициатором выступил умудренный опытом наставник – сперва он начал кидаться в оппонента своего подопечного предметом, похожим на мяч для американского футбола, а когда тот пинком вынес данный «снаряд» за пределы настила, подошел к молодому человеку вплотную для серьезного разговора. Представитель тренерского штаба был недоволен судейским решением, но почему-то решил высказать это не арбитру, а атлету. После пары ласковых слов, которых на записи, к счастью, не слышно, специалист неожиданно нанес спортсмену хук слева, а затем и справа. Борец только пошатнулся и не успел ничем ответить — тут же между ним и агрессором вырос принявшийся разнимать оппонентов представитель охраны. Однако не тут-то было. Сзади к пострадавшему подкрался его соперник по поединку и несколь